### Time to get your hands dirty! The irreducible mass of black holes.

This is an example that comes straight out of my research in black-hole physics (and was first worked out by Cecilia Maria Fabbri in her BSc thesis).

Consider a rotating black hole. In [1969](https://ui.adsabs.harvard.edu/abs/1969NCimR...1..252P/abstract), Roger Penrose (now [Nobel Prize in Physics](https://www.nobelprize.org/prizes/physics/2020/penrose/facts/) for the theory of black hole) discovered that one can extract energy from a rotating black hole:

![](https://inspirehep.net/files/268d28446b405e134199d84f67c23792)

(figure from [Harada, Kimura 2014](https://ui.adsabs.harvard.edu/abs/2014CQGra..31x3001H/abstract)).

Maybe some of you has seen this already in your GR class? In any case, a rotating black-hole has a region called ergosphere such that infalling particles have negative energy. Penrose imagined a particle that enters that region and decays: half will fall in and half will escape to infinity, carring some energy with it. In a nutshell, we've extracted energy=mass from a black hole! It turns out that one also has to extract angular momentum, not just energy (and here you'll need some GR do to this calculation, see e.g. the famous textbook by [Misner, Thorne and Weeler](https://press.princeton.edu/books/hardcover/9780691177793/gravitation). But angular momentum runs our first, such that a repeated series of Penrose processes leaves behind a non-rotating BH with a remnant mass which is called "irreducible". This is a famous result by [Christodoulou 1970](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.25.1596)) and is closely related to the even more famous area law by [Hawking (1971)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.26.1344) (which in turns is the foundation of black-hole thermodynamics).


For a BH of mass $M$ spin $\chi\in [0,1]$, the irreducible mass is 
\begin{equation}
M_{\rm irr} = %\sqrt{\frac{M^2+\sqrt{M^4-S^2}}{2}} = 
 M \sqrt{\frac{1+\sqrt{1-\chi^2}}{2}}
\label{mirrdef}
\end{equation}
For convenience, let us define $f= M_{\rm irr}/M$

Suppose you have repeated measurment of $M$ and $\chi$. This is just because we're still frequentists for now. When we put our Bayesian hat, these distributions will be the *posteriors* from a single measurement.

Let's assume that

- the spin $\chi$ is uniformly distributed between 0 and 1.
- the mass $M$ is normally distributed with mean $\mu$ and standard deviation $\sigma$.

These are simplistic assumption that, however, are not too far from what we are now measuring with gravitational-wave detectors. It's a toy model, but not a totally unreasonable one.

**Tasks**:

1. Compute the pdf of $f$. *Hint*: we saw how to do this in a previous lecture. The answer is:
\begin{equation}
\pi(f) = 2\frac{2 f^2-1}{\sqrt{1 - f^2}}\,.
\end{equation}

2. Use [this theorem](https://en.wikipedia.org/wiki/Distribution_of_the_product_of_two_random_variables) to compute the pdf of $M_{\rm irr}$. The answer is: 
\begin{equation}
p(M_{\rm irr})  = \frac{\sqrt{2/\pi}}{\sigma} \int_{1/\sqrt{2}}^1 \exp\left[{-\frac{(M_{\rm irr}/f -\mu)^2}{2\sigma^2}}\right]  \frac{2 f^2-1}{f \sqrt{1 - f^2}}  \,\, {\rm d}f\,. \label{pmirr}
\end{equation}
 
3. You can normalize all quantities with a dimension of mass by $\mu$. Just set $\mu=1$ in your code and remember that all masses are measured in units of $\mu$.

4. Sample $M_{\rm irr}$ from its pdf. *Hint*: check the sampling methods we saw earlier in the course. *Even better hint*: you don't even need that! So easy! Think about it.

5. Compute the KS distance between ${M_{\rm irr}}$ and ${f}$ as a function of $\sigma$. Discuss the limits at large and small values of $\sigma$. 

6. Compute the KS distance between ${M_{\rm irr}}$ and $M$ as a function of $\sigma$. Discuss the limits at large and small values of $\sigma$.

7. You should find that

\begin{alignat}{10}
p\left(M_{\rm irr}\right)  &\simeq \frac{\pi(M_{\rm irr}/\mu)}{\mu} \qquad&&{\rm for}\quad {\sigma}\ll \mu\,, \label{limit1}\\
p(M_{\rm irr})  &\simeq \mathcal{N}(\mu,\sigma) \qquad && {\rm for}\quad \sigma\gg \mu \,. \label{limit2}
\end{alignat}

### Time to get your hands dirty! The irreducible mass of black holes.

Let's go back to our example from above. Using samples describing the irreducible mass of black holes, fit a KDE and check it against the analytic result (i.e. that integral in $df$).  Let's pick $\sigma=0.02$